<a href="https://colab.research.google.com/github/ZinoTAT/Machine-Learning/blob/master/pytorch/practices/resnet%E6%A8%A1%E5%9E%8B%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

自己动手写一个resnet
![resnet](https://github.com/ZinoTAT/Machine-Learning/blob/master/images/resnet.png?raw=true)

In [1]:
import torch
import torch.nn as nn
print(torch.__version__)

1.4.0


为方便后续使用，封装一下接下来将要使用的3 * 3卷积和1 * 1卷积

In [0]:
def conv3x3(in_channels, out_channels, stride=1, padding=1):
  return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=padding, bias=False) #bias设置为false是为了在有bn层的情况下节省参数
def conv1x1(in_channels, out_channels, stride=1, padding=0):
  return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=padding, bias=False) #bias设置为false是为了在有bn层的情况下节省参数

50层以下使用的BasicBlock

In [0]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_channels, out_channels, stride=1, downsample=None, norm_layer=None):
    super(BasicBlock, self).__init__()
    if norm_layer is None:
      norm_layer = nn.BatchNorm2d
    self.conv1 = conv3x3(in_channels, out_channels, stride)
    self.bn1 = norm_layer(out_channels)
    self.relu = nn.ReLU(inplace=True)
    self.conv2 = conv3x3(out_channels, out_channels)
    self.bn2 = norm_layer(out_channels)
    self.downsample = downsample
    self.stride = stride
  def forward(self, x):
    identity = x

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)

    x = self.conv2(x)
    x = self.bn2(x)

    if self.downsample is not None:
      identity = self.downsample(identity) #如果输入的x和输出的x存在维度变换，则要先将维度进行统一再相加

    x += identity
    x = self.relu(x)

    return x

50层以上使用的BottleBlock

In [0]:
class BottleBlock(nn.Module):
  expansion = 4

  def __init__(self, in_channels, channels, stride=1, downsample=None, norm_layer=None):
    super(BottleBlock, self).__init__()
    if norm_layer is None:
      norm_layer = nn.BatchNorm2d
    self.conv1 = conv1x1(in_channels, channels)
    self.bn1 = norm_layer(channels)
    self.conv2 = conv3x3(channels, channels, stride=stride)
    self.bn2 = norm_layer(channels)
    self.conv3 = conv1x1(channels, channels * self.expansion)
    self.bn3 = norm_layer(channels * self.expansion)
    self.relu = nn.ReLU(inplace=True)
    self.downsample = downsample
    self.stride = stride
  def forward(self, x):
    identity = x

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)

    x = self.conv3(x)
    x = self.bn3(x)
    
    if self.downsample is not None:
      identity = self.downsample(identity)
    
    x += identity
    x = self.relu(x)

    return x


In [0]:
class ResNet(nn.Module):
  def __init__(self, name, block, layers, num_class=1000, norm_layer=None):
    super(ResNet, self).__init__()
    if norm_layer is None:
      norm_layer = nn.BatchNorm2d
    self._norm_layer = norm_layer

    self.in_channels = 64
    self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
    self.bn1 = norm_layer(self.in_channels)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.layer1 = self._make_layer(block, 64, layers[0])
    self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
    self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
    self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
    self.AdaptiveAvgPool2d = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512 * block.expansion, num_class)
    self.relu = nn.ReLU(inplace=True)
    self.name = name
  
  def _make_layer(self, block, channels, blocks, stride=1):
    
    downsample = None
    norm_layer = self._norm_layer
    if stride != 1 or self.in_channels != channels * block.expansion:
      downsample = nn.Sequential(
          conv1x1(in_channels=self.in_channels, out_channels=channels * block.expansion, stride=stride),
          norm_layer(channels * block.expansion)
      )
    layers = []
    layers.append(block(self.in_channels, channels, stride, downsample, norm_layer))
    self.in_channels = channels * block.expansion
    for _ in range(1, blocks):
      layers.append(block(self.in_channels, channels,norm_layer=norm_layer))
    return nn.Sequential(*layers)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.AdaptiveAvgPool2d(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)
    return x



In [6]:
def resnet18():
  return ResNet('resnet18', BottleBlock, [2, 2, 2, 2])
def resnet50():
  return ResNet('resnet50', BottleBlock, [3, 4, 6, 3])
rn18 = resnet18()
rn50 = resnet50()
x = torch.randn([1, 3, 224, 224])
y1 = rn18.forward(x)
y2 = rn50.forward(x)
print(y1.shape, y2.shape)

torch.Size([1, 1000]) torch.Size([1, 1000])
